# How to Optimize ARIMA Model Fits
> "A gentle guide to optimizing ARIMA models for time series analysis."

- toc: false
- branch: master
- badges: true 
- permalink: /time-series-optimize-ARIMA/
- comments: false
- hide: false
- categories: [Beginner, Time Series Analysis]

In [ ]:
#hide
import warnings
!pip install statsmodels
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', category=ConvergenceWarning)
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import plotly.express as px # for visualization 
import plotly.graph_objects as go
from statsmodels.tsa.arima.model import ARIMA
from plotly.subplots import make_subplots

'''Here we import our data, and take two thirds of it for model training'''
raw_data_DF = pd.read_csv("monthly-sunspots.txt", parse_dates=[0])

train_set_size = int(len(raw_data_DF.index) * 0.66)

df = raw_data_DF.iloc[:train_set_size]
timeSeries = df['Sunspots']


You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
#hide

pVal = 37
qVal = 6
dVal = 0


model = ARIMA(timeSeries, order=(pVal,dVal,qVal))
result = model.fit()

df = raw_data_DF
timeSeries = df['Sunspots']

predictedVals = result.get_prediction(start=0,end=len(timeSeries))

df['estimate'] = predictedVals.summary_frame()['mean']

fig = px.line()
fig.add_trace(go.Scatter(
                    name='Real Data', 
                    x=df['Month'], 
                    y=df['Sunspots'],
                    mode='lines'
                    ))

fig.add_trace(go.Scatter(
                    name='Model Estimate', 
                    x=df['Month'], 
                    y=df['estimate'],
                    mode='lines'
                    ))

fig.add_trace(go.Scatter(
                    name='Train/Test Split',
                    x=[df['Month'][train_set_size],df['Month'][train_set_size]],
                    y=[0,300],
                    line_width=3,
                    line_dash="dash", 
                    line_color='black'
                    ))





---

[My last post]({% post_url 2022-01-24-timeSeriesIntroARIMA %}) ended with a reasonably successful fit of some time series data using the ARIMA model. To start this post off, I've applied the exact same method to fit a different dataset. The dataset we used in the previous post was pretty small, and didn't leave many options for train/test protocols. This new dataset is quite a bit longer, just to help illustrate how some of these techniques work. The dataset we'll be using is [the monthly sunspot dataset from Jason Brownlee's GitHub.](https://github.com/jbrownlee/Datasets/) The resulting fit from using the method found in my previous post is shown below.





In [ ]:
#hide_input
fig.show()

That looks to be a pretty good fit, recall that ARIMA is only intended for short-term forecasting, but we haven't actually quantified how good the fit is in any way. In order to do that, we typically use something called the Root Mean Squared Error (RMSE).

# Root Mean Squared Error

The RMSE of a fit is determined according to the following formula:

$RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n}\big(Y_i - \hat{Y}_i\big)^2}$

Where $Y_i$ represents the experimental data, $\hat{Y}_i$ represents the model estimations. This parameter should only be calculated for the portion of the data that the model hasn't seen, also called the *validation data set*. So, let's grab that data set again, fit it the same way as last time, and do some math.

In [ ]:
import pandas as pd
import plotly.express as px # for visualization 
import plotly.graph_objects as go
from statsmodels.tsa.arima.model import ARIMA
from plotly.subplots import make_subplots

# We load the data into a pandas DataFrame
raw_data_DF = pd.read_csv("monthly-sunspots.txt", parse_dates=[0])

# Then we split it into training and test sets
train_set_size = int(len(raw_data_DF.index) * 0.66)
df = raw_data_DF.iloc[:train_set_size]
timeSeries = df['Sunspots']

# These are the ideal ARIMA fit parameters determined according to the method from my previous post
pVal = 37
qVal = 6
dVal = 0

# Then we fit the model.
model = ARIMA(timeSeries, order=(pVal,dVal,qVal))
result = model.fit()

In [ ]:
# We get predictions from our fitted model
df = raw_data_DF
timeSeries = df['Sunspots']
predictedVals = result.get_prediction(start=0,end=len(timeSeries))
df['estimate'] = predictedVals.summary_frame()['mean']

# And we calculate the RMSE from the portion of validation portion of the DataFrame
validationDF = df.iloc[train_set_size:]
RMSE = (((validationDF["Sunspots"] - validationDF["estimate"]) ** 2).mean() ) ** 0.5
print("RMSE = ", RMSE)

RMSE =  52.377481932252316


The RMSE is just over 50. Is that good? Is it bad? The RMSE is convenient because its raw value is in the same units as the value we are predicting, which makes it easy to visualize the amount of error in our predictions. That makes it clear that for this metric lower is better, but we don't have anything to compare it to. We're also not totally certain that we've chosen the best parameters for the model, and we've only trained and tested the model on one dataset. That's a lot of uncertainty, and if we want to put something like this into production we're going to have to do better.


# Walk Forward Validation

It would be helpful if we had more datasets to train and validate this data on, but data can be difficult and expensive to collect. Gathering enough data to make a decent model can also take a lot of time. That's where Walk Forward Validation comes in. Previously, we split our dataset into a single training and validation dataset.

In [ ]:
#hide_input
df_train = raw_data_DF.iloc[:train_set_size]
df_valid = raw_data_DF.iloc[train_set_size:]

fig = px.line()
fig.add_trace(
    go.Scatter(
    name='Training Set', 
    x=df_train['Month'], 
    y=df_train['Sunspots'],
    mode='lines'
    ))

fig.add_trace(
    go.Scatter(
    name='Validation Set', 
    x=df_valid['Month'], 
    y=df_valid['Sunspots'],
    mode='lines'
    ))
fig.show()

                    

If we get a little bit smarter though, we can split it into multiple sets, treating each individual subset as an entirely different data set. We have to be careful when doing this with time series data, and make sure that no information from the future leaks into the past when we're training the model. No one wants another Back to the Future 3. Before we do anything though, lets save the last 10% of our dataset for use in a final test to check the quality of our fit.

In [ ]:
from plotly.subplots import make_subplots

# We hold back 10% of the data for use in a final validation
holdback_DF_size = int(len(raw_data_DF.index) * 0.10)
walk_forward_DF = raw_data_DF[:-holdback_DF_size]

# This is a fun bit of code that determines the size of the training and validation sets necessary to break our dataset into
# num_data_sets different datasets.
num_data_sets = 6
validation_set_size = int(len(walk_forward_DF.index)/(num_data_sets+2))
train_set_size = validation_set_size * 2


fig = make_subplots(rows=6, cols=1, shared_xaxes=True)


# We walk through the data, and break it up into num_data_sets individual train/test splits.
# The resulting splits have been plotted below.
train_set_start = 0
train_set_end = train_set_start + train_set_size
validation_set_start = train_set_end
validation_set_end = validation_set_start + validation_set_size
data_set_number = 1
while validation_set_end < len(walk_forward_DF.index):
 
    df_train = walk_forward_DF.iloc[train_set_start:train_set_end]
    df_valid = walk_forward_DF.iloc[validation_set_start:validation_set_end]

    fig.add_trace(
        go.Scatter(
        name= "Training Set",
        legendgroup="group" + str(data_set_number),
        legendgrouptitle_text=str(data_set_number),
        x=df_train['Month'], 
        y=df_train['Sunspots'], 
        mode='lines'),
        row=data_set_number, col=1
    )

    fig.add_trace(
        go.Scatter(
        name= "Validation Set",
        legendgroup="group" + str(data_set_number),
        legendgrouptitle_text=str(data_set_number),
        x=df_valid['Month'], 
        y=df_valid['Sunspots'], 
        mode='lines'),
        row=data_set_number, col=1
    )



    train_set_start = train_set_start + validation_set_size
    train_set_end = train_set_start + train_set_size
    validation_set_start = train_set_end
    validation_set_end = validation_set_start + validation_set_size
    data_set_number = data_set_number + 1


fig.update_xaxes(title_text="Date", row=6,col=1)
fig.update_layout(height=800)
fig.show()

Now we have six individual datasets to train and test on! This is a pretty small dataset, and we want to make sure each individual dataset is large enough to be useful, so we can't go too much smaller than that. In practice, a dataset can be broken into as many or as few test/validation pairs as you need so long as you don't make them too small!

Let's use this in combination with the RMSE we just learned about to get a better sense of how our model is doing! Remember, we need to fit on each training set as we loop through the data and then determine the RMSE from the validation set. 

In [ ]:
# These are the fitting parameters determined from the original method.
pVal = 37
qVal = 6
dVal = 0


fig = make_subplots(rows=6, cols=1, shared_xaxes=True)

RMSE_list = []

num_data_sets = 6
validation_set_size = int(len(walk_forward_DF.index)/(num_data_sets+2))
train_set_size = validation_set_size * 2

# Basically the same code as above, but this time we fit our ARIMA model on each training set and test it on the validation set.
# The resulting fits have been plotted alongside the train/test splits shown previously.
train_set_start = 0
train_set_end = train_set_start + train_set_size
validation_set_start = train_set_end
validation_set_end = validation_set_start + validation_set_size
data_set_number = 1
while validation_set_end < len(walk_forward_DF.index):
 
    df = walk_forward_DF.iloc[train_set_start:validation_set_end]
    df_train = walk_forward_DF.iloc[train_set_start:train_set_end]
    df_valid = walk_forward_DF.iloc[validation_set_start:validation_set_end]

    train_time_series = df_train['Sunspots']

    model = ARIMA(train_time_series, order=(pVal,dVal,qVal))
    result = model.fit()

    predictedVals = result.get_prediction(start=0,end=validation_set_end - train_set_start)

    df['estimate'] = predictedVals.summary_frame()['mean']

    validationDF = df.iloc[train_set_size:]
    RMSE = (((validationDF["Sunspots"] - validationDF["estimate"]) ** 2).mean() ) ** 0.5
    RMSE_list.append(RMSE)

    fig.add_trace(
        go.Scatter(
        name= "Training Set",
        legendgroup="group" + str(data_set_number),
        legendgrouptitle_text=str(data_set_number),
        x=df_train['Month'], 
        y=df_train['Sunspots'], 
        mode='lines'),
        row=data_set_number, col=1
    )

    fig.add_trace(
        go.Scatter(
        name= "Validation Set",
        legendgroup="group" + str(data_set_number),
        legendgrouptitle_text=str(data_set_number),
        x=df_valid['Month'], 
        y=df_valid['Sunspots'], 
        mode='lines'),
        row=data_set_number, col=1
    )

    fig.add_trace(
        go.Scatter(
        name= "Model Prediction",
        legendgroup="group" + str(data_set_number),
        legendgrouptitle_text=str(data_set_number),
        x=df['Month'], 
        y=df['estimate'], 
        mode='lines'),
        row=data_set_number, col=1
    )


    train_set_start = train_set_start + validation_set_size
    train_set_end = train_set_start + train_set_size
    validation_set_start = train_set_end
    validation_set_end = validation_set_start + validation_set_size
    data_set_number = data_set_number + 1


# Since we have a sample of fitting results, we can calculate a confidence interval for the RMSE values
mean_RMSE = sum(RMSE_list)/len(RMSE_list)
z_val = 1.96 # This is the z value for a 95% confidence interval
conf_interval = z_val * np.std(RMSE_list)/np.sqrt(len(RMSE_list))

fig.update_xaxes(title_text="Date", row=6,col=1)
fig.update_layout(height=800)
fig.show()

print("RMSE: ", mean_RMSE, " +/- ", conf_interval)

RMSE:  41.16820390420083  +/-  11.071244210278234


The confidence interval we see above, calculated using the various walk forward train/test splits we just made, gives us a better idea of our model performance for this data set. Because it gives us a range of values that we might be able to expect for the RMSE, we can evaluate for ourselves if the amount of error in that range is acceptable for whatever application we have in mind. The model fits for each of the train/test splits are also shown, to illustrate how this value describes the error in the fits.

If we're willing to commit to a very long computation, we can take our fitting a step further.

# Grid Search and Walk Forward for ARIMA Optimization

To make our parameter estimation even more robust, we can perform a grid search of the parameters surrounding our best estimates. We perform a walk forward train/test validation of all the different parameter combinations, and record their mean RMSE values. Once a walk forward validation has been performed using each parameter combination, we take the paramater combination with the best mean RMSE.

In [ ]:
# We hold back 10% of the data for use in a final validation
holdback_DF_size = int(len(raw_data_DF.index) * 0.10)
walk_forward_DF = raw_data_DF[:-holdback_DF_size]

num_data_sets = 6

validation_set_size = int(len(walk_forward_DF.index)/(num_data_sets+2))
train_set_size = validation_set_size * 2

RMSE_results = dict()

#This describes the values we will be testing for our ARIMA model
for pVal in range (34,41):
    for qVal in range(4,9):
        for dVal in range(0,2):

            RMSE_list = []

            train_set_start = 0
            train_set_end = train_set_start + train_set_size
            validation_set_start = train_set_end
            validation_set_end = validation_set_start + validation_set_size
            while validation_set_end < len(walk_forward_DF.index):

                df = walk_forward_DF.iloc[train_set_start:validation_set_end]
                df_train = walk_forward_DF.iloc[train_set_start:train_set_end]
                df_valid = walk_forward_DF.iloc[validation_set_start:validation_set_end]

                model = ARIMA(df_train['Sunspots'], order=(pVal,dVal,qVal))
                result = model.fit()

                predictedVals = result.get_prediction(start=0,end=validation_set_end - train_set_start)

                df['estimate'] = predictedVals.summary_frame()['mean']

                validationDF = df.iloc[train_set_size:]

                RMSE = (((validationDF["Sunspots"] - validationDF["estimate"]) ** 2).mean() ) ** 0.5
                RMSE_list.append(RMSE)

                train_set_start = train_set_start + validation_set_size
                train_set_end = train_set_start + train_set_size
                validation_set_start = train_set_end
                validation_set_end = validation_set_start + validation_set_size

            mean_RMSE = sum(RMSE_list)/len(RMSE_list)
            RMSE_results[mean_RMSE] = [pVal,dVal,qVal]
            
min_key = np.inf

for key in RMSE_results:
    if key < min_key:
        min_key = key



print("Best RMSE: ", min_key)
print("Best p,d,q: ", RMSE_results[min_key])

Best RMSE:  39.41639690453025
Best p,d,q:  [36, 1, 5]


# Grid Search Results

The above test determined that our optimum parameters are p = 36, d = 1, and q = 5. It gave us a slightly better RMSE, but that calculation took about 18 hours. Let's use our new optimally determined parameters with the walk forward method, and generate a distribution for the RMSE in using the new parameters. In principle this could have been done as part of the previous step, but for the sake of illustrating the process we'll do it here.

In [ ]:
# These are the fitting parameters determined from our optimization
pVal = 36
qVal = 5
dVal = 1

holdback_DF_size = int(len(raw_data_DF.index) * 0.10)
walk_forward_DF = raw_data_DF[:-holdback_DF_size]

fig = make_subplots(rows=6, cols=1, shared_xaxes=True)

RMSE_list = []

num_data_sets = 6
validation_set_size = int(len(walk_forward_DF.index)/(num_data_sets+2))
train_set_size = validation_set_size * 2

# Basically the same code as above, but this time we fit our ARIMA model on each training set and test it on the validation set.
# The resulting fits have been plotted alongside the train/test splits shown previously.
train_set_start = 0
train_set_end = train_set_start + train_set_size
validation_set_start = train_set_end
validation_set_end = validation_set_start + validation_set_size
data_set_number = 1

while validation_set_end < len(walk_forward_DF.index):
 
    df = walk_forward_DF.iloc[train_set_start:validation_set_end]
    df_train = walk_forward_DF.iloc[train_set_start:train_set_end]
    df_valid = walk_forward_DF.iloc[validation_set_start:validation_set_end]

    train_time_series = df_train['Sunspots']

    model = ARIMA(train_time_series, order=(pVal,dVal,qVal))
    result = model.fit()

    predictedVals = result.get_prediction(start=0,end=validation_set_end - train_set_start)

    df['estimate'] = predictedVals.summary_frame()['mean']

    validationDF = df.iloc[train_set_size:]
    RMSE = (((validationDF["Sunspots"] - validationDF["estimate"]) ** 2).mean() ) ** 0.5
    RMSE_list.append(RMSE)

    fig.add_trace(
        go.Scatter(
        name= "Training Set",
        legendgroup="group" + str(data_set_number),
        legendgrouptitle_text=str(data_set_number),
        x=df_train['Month'], 
        y=df_train['Sunspots'], 
        mode='lines'),
        row=data_set_number, col=1
    )

    fig.add_trace(
        go.Scatter(
        name= "Validation Set",
        legendgroup="group" + str(data_set_number),
        legendgrouptitle_text=str(data_set_number),
        x=df_valid['Month'], 
        y=df_valid['Sunspots'], 
        mode='lines'),
        row=data_set_number, col=1
    )

    fig.add_trace(
        go.Scatter(
        name= "Model Prediction",
        legendgroup="group" + str(data_set_number),
        legendgrouptitle_text=str(data_set_number),
        x=df['Month'], 
        y=df['estimate'], 
        mode='lines'),
        row=data_set_number, col=1
    )


    train_set_start = train_set_start + validation_set_size
    train_set_end = train_set_start + train_set_size
    validation_set_start = train_set_end
    validation_set_end = validation_set_start + validation_set_size
    data_set_number = data_set_number + 1



# Since we have a sample of fitting results, we can calculate a confidence interval for the RMSE values
mean_RMSE = sum(RMSE_list)/len(RMSE_list)
z_val = 1.96 # This is the z value for a 95% confidence interval
conf_interval = z_val * np.std(RMSE_list)/np.sqrt(len(RMSE_list))

fig.update_xaxes(title_text="Date", row=6,col=1)
fig.update_layout(height=800)
fig.show()

print("RMSE: ", mean_RMSE, " +/- ", conf_interval)

RMSE:  39.41639690453025  +/-  11.408986555476803


The confidence interval is about the same as our previous example. Let's compare the confidence intervals for our original and optimized models.

In [ ]:
import plotly.graph_objects as go

RMSE_original = 41.16820390420083
error_original =  11.071244210278234

RMSE_opt = 39.41639690453025
error_opt = 11.408986555476803

df = pd.DataFrame({
    "Method": ["Original", "Optimization"],
    "Value": [RMSE_original,RMSE_opt],
    "Error": [error_original,error_opt]
})


fig = go.Figure()

fig.add_trace(go.Bar(
            x = df["Method"], 
            y = df["Value"],
            error_y = dict(
                type = 'data',
                array = df["Error"],
                visible = True
            ),
))

fig.update_layout(
    title = "Confidence interval comparison",
    width=400
    )

fig.update_yaxes(title="RMSE")

So the actual means were approximately 41.2 for the original method, and 39.4 for the optimization. Those are almost negligibly different, especially when you consider that their confidence intervals are almost entirely overlapped. In order to directly compare the two fits, we should actually employ both RMSE and the [Akaike Information Criterion (AIC)](https://en.wikipedia.org/wiki/Akaike_information_criterion). Recall that these should be calculated on data that the model has not been trained on. AIC essentially evaluates a models performance on a dataset while discouraging increased model complexity. Each model tested on the dataset will have a distinct AIC value, with lower values indicating better model performance.

There are many ways to calculate the AIC parameter of a fit, here we will use the residual sum of squares (RSS) as the likelihood parameter according to the following equations:

$AIC = 2k - 2 \mathrm{ln}(\hat{L})=2k - 2 \mathrm{ln}(RSS)$

$RSS =  \sum_{i=1}^{n}\big(Y_i - \hat{Y}_i\big)^2$

Where $k$ is the number of fitting parameters for the model, $\hat{L}$ is the likelihood function, $Y_i$ are the observed data points, and $\hat{Y_i}$ are the estimated data points.

In [ ]:
# Original model parameters
pVal = 37
qVal = 6
dVal = 0

# Optimal model parameters
pVal_opt = 36
qVal_opt = 5
dVal_opt = 1

# We hold back 10% of the data for use in a final validation
holdback_DF_size = int(len(raw_data_DF.index) * 0.10)

# Make a dataframe that doesn't include the holdback data
df = raw_data_DF[:-holdback_DF_size]
timeSeries = df['Sunspots']

# Train both models on this data.
model = ARIMA(timeSeries, order=(pVal,dVal,qVal))
result = model.fit()

model_opt = ARIMA(timeSeries, order=(pVal_opt,dVal_opt,qVal_opt))
result_opt = model_opt.fit()

# Make a dataframe that includes the holdback data
df = raw_data_DF

# Use both models to predict the holdback data
predicted_vals = result.get_prediction(start=0,end=len(df.index))
predicted_vals_opt = result_opt.get_prediction(start=0,end=len(df.index))

df['estimate'] = predicted_vals.summary_frame()['mean']
df['estimate_opt'] = predicted_vals_opt.summary_frame()['mean']

# Make a dataframe from only the holdback data
validation_DF = df[-holdback_DF_size:]

# Determine the RMSE for the holdback portion, then plot the data alongside the model fits
RMSE = (((validation_DF["Sunspots"] - validation_DF["estimate"]) ** 2).mean() ) ** 0.5
RMSE_opt = (((validation_DF["Sunspots"] - validation_DF["estimate_opt"]) ** 2).mean() ) ** 0.5

k_orig = pVal + qVal + 1
k_opt = pVal_opt + qVal_opt + 1

RSS_orig = sum((validation_DF["Sunspots"] - validation_DF["estimate"]) ** 2)
RSS_opt = sum((validation_DF["Sunspots"] - validation_DF["estimate_opt"]) ** 2)

AIC_orig = 2 * k_orig - 2 * np.log(RSS_orig)
AIC_opt = 2 * k_opt - 2 * np.log(RSS_opt)

fig = px.line()
fig.add_trace(go.Scatter(
                    name='Real Data', 
                    x=validation_DF['Month'], 
                    y=validation_DF['Sunspots'],
                    mode='lines'
                    ))

fig.add_trace(go.Scatter(
                    name='Original Estimate', 
                    x=validation_DF['Month'], 
                    y=validation_DF['estimate'],
                    mode='lines'
                    ))

fig.add_trace(go.Scatter(
                    name='Optimized Estimate', 
                    x=validation_DF['Month'], 
                    y=validation_DF['estimate_opt'],
                    mode='lines'
                    ))

fig.show()

# Conveniently, statsmodels offers us a very simple way of determing the aic of our models.
print("RMSE, AIC Original Model: ", RMSE,AIC_orig)
print("RMSE, AIC Optimized Model: ", RMSE_opt,AIC_opt)

RMSE, AIC Original Model:  45.81924188577674 61.41736933251867
RMSE, AIC Optimized Model:  45.82037338502683 57.41727055434315


The RMSE and AIC abserved when testing both our original and optimized models on our holdback data can be seen above. The RMSE for our original model is ever so slightly better than for our optimized model, with values of 45.819 and 45.820 respectively. The AIC has values of 61.4 for our original model and 57.4 for our optimized model, indicating that the optimal model is a better fit! This is because, while the original model may be offering a very slightly better fit in terms of absolute error, the optimized model has fewer fitting parameters. Adding extra fitting parameters with negligible model performance is discouraged using AIC, which results in the optimized model having a lower AIC value.

Finding the optimized model parameters ended up taking 18 hours though, and resulted in marginally better perfomance. It could be argued that, for this application, the original model is just as performant and without requiring a lengthy optimization.

# Up Next

We'll either be taking a look at using Recurrent Neural Networks and Long Short-Term Memory to try to predict cryptocurrency prices, or using SHapley Additive exPlanations to define the most important input variables for complex machine learning models.